In [39]:
MOVIE_GENRES = [
    'Drama', 'Comedy', 'Documentary', 'Romance', 'Action',
    'Crime', 'Thriller', 'Horror', 'Adventure', 'Mystery'
]

In [38]:
import pandas as pd
df = pd.read_parquet("../data/movies_with_release_dates_status_filtered.parquet") # Adjust path if needed
print(df['fetch_status'].value_counts(dropna=False))
success_df = df[df['fetch_status'] == 'SUCCESS']
print(f"Number of SUCCESS: {len(success_df)}")
print(success_df['release_date_full'].notna().sum()) # SUCCESS with a date string

fetch_status
None                         694605
SUCCESS                       16261
ERROR_GENERIC_MAX_RETRIES      2790
NOT_FOUND_FIND                  325
NO_DATE_FIELD                    34
Name: count, dtype: int64
Number of SUCCESS: 16261
16261


In [21]:
import pandas as pd
from tmdbv3api import TMDb, Movie, Find # Ensure these are imported
import time
import os
import multiprocessing
from functools import partial

In [30]:
# Read /Users/frederikreimert/Library/CloudStorage/OneDrive-DanmarksTekniskeUniversitet/Kandidat_DTU/2025F/02806 Social Data Analysis and Visualization/final_project/data/movies_with_release_dates_status_filtered.parquet

df = pd.read_parquet("../data/movies_with_release_dates_status_filtered.parquet") # Adjust path if needed

In [34]:
df.columns

Index(['title_id', 'title', 'original_title', 'release_year',
       'runtime_minutes', 'genre', 'imdb_rating', 'vote_count',
       'production_country', 'release_date_full', 'fetch_status'],
      dtype='object')

In [37]:
MOVIE_GENRES = [
    'Drama', 'Comedy', 'Documentary', 'Romance', 'Action',
    'Crime', 'Thriller', 'Horror', 'Adventure', 'Mystery'
]

In [33]:
# See number of release_date_full
print(df['release_date_full'].notna().sum()) # SUCCESS with a date string

14692


In [44]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px # For color scales if needed for other things

# --- Configuration: Import from plot_style.py or use fallbacks ---
try:
    from plot_style import MOVIE_GENRES, GENRE_COLOR_MAP
except ImportError:
    print(
        "Warning: plot_style.py not found or MOVIE_GENRES/GENRE_COLOR_MAP "
        "not defined. Using fallback definitions."
    )
    MOVIE_GENRES = [
        'Drama', 'Comedy', 'Documentary', 'Romance', 'Action',
        'Crime', 'Thriller', 'Horror', 'Adventure', 'Mystery'
    ]
    _COLORS_HEX_FALLBACK = [
        '#003f5c', '#f9a602', '#7a7a7a', '#ff7b9c', '#ef562f',
        '#2f4f4f', '#008080', '#8b0000', '#556b2f', '#6a0dad'
    ]
    if len(MOVIE_GENRES) > len(_COLORS_HEX_FALLBACK):
        _COLORS_HEX_FALLBACK = px.colors.qualitative.Plotly[:len(MOVIE_GENRES)]
    GENRE_COLOR_MAP = dict(zip(MOVIE_GENRES, _COLORS_HEX_FALLBACK[:len(MOVIE_GENRES)]))

# --- Constants for polar plot calculations ---
N_WEEKS = 52
DEGREES_PER_WEEK = 360.0 / N_WEEKS
COMBINED_VIEW_LABEL = "All Genres (Stacked)"

# --- Season Definitions (Approximate for Northern Hemisphere) ---
SEASONS = [
    {"name": "Winter", "start_week": 51, "end_week": N_WEEKS, "color": "rgba(173, 216, 230, 0.15)"},
    {"name": "Winter", "start_week": 1, "end_week": 11, "color": "rgba(173, 216, 230, 0.15)"},
    {"name": "Spring", "start_week": 12, "end_week": 24, "color": "rgba(144, 238, 144, 0.15)"},
    {"name": "Summer", "start_week": 25, "end_week": 37, "color": "rgba(255, 255, 153, 0.2)"},
    {"name": "Autumn", "start_week": 38, "end_week": 50, "color": "rgba(255, 192, 122, 0.15)"}
]

# --- Step 1: Load and Prepare Data ---
def create_sample_dataframe():
    num_records = 4000 # Increased for better combined view
    all_possible_genres_for_sampling = MOVIE_GENRES + ['Sci-Fi', 'Fantasy', 'Animation', 'Family', 'War', 'Music', 'Biography']
    
    genres_list = []
    for _ in range(num_records):
        num_genres_for_movie = np.random.choice([1, 2, 3], p=[0.6, 0.3, 0.1])
        selected_genres = np.random.choice(all_possible_genres_for_sampling, num_genres_for_movie, replace=False)
        genres_list.append(", ".join(selected_genres))

    data = {
        'title_id': range(1, num_records + 1),
        'genre': genres_list,
        'release_date_full': pd.Series(pd.date_range(start='2022-01-01', end='2023-12-31', freq='D'))
                               .sample(num_records, replace=True).values,
    }
    df_temp = pd.DataFrame(data)
    num_na = int(num_records * 0.05)
    na_indices = np.random.choice(df_temp.index, num_na, replace=False)
    df_temp.loc[na_indices, 'release_date_full'] = pd.NaT
    return df_temp

df = create_sample_dataframe()
# df = pd.read_csv('your_movie_data.csv') # Your actual data loading

df['release_date_full'] = pd.to_datetime(df['release_date_full'], errors='coerce')
df.dropna(subset=['release_date_full'], inplace=True)
df['release_week'] = df['release_date_full'].dt.isocalendar().week.astype(int)
df.loc[df['release_week'] > N_WEEKS, 'release_week'] = N_WEEKS

df_exploded = df.assign(genre=df['genre'].astype(str).str.strip().str.split(', ')).explode('genre')
df_exploded['genre'] = df_exploded['genre'].str.strip()
df_focus_genres = df_exploded[df_exploded['genre'].isin(MOVIE_GENRES)].copy()

# --- 1.A: Prepare data for individual genre plots AND stacked combined plot ---
all_weeks_series = pd.Series(range(1, N_WEEKS + 1), name='release_week')
if not df_focus_genres.empty and MOVIE_GENRES:
    weekly_counts_per_genre = df_focus_genres.groupby(['genre', 'release_week']).size().reset_index(name='movie_count')
    
    # Create a template for all genre-week combinations to ensure all weeks are present for all genres
    all_genres_weeks_template = pd.MultiIndex.from_product(
        [MOVIE_GENRES, all_weeks_series], names=['genre', 'release_week']
    ).to_frame(index=False)
    
    prepared_data_for_traces = pd.merge(
        all_genres_weeks_template, weekly_counts_per_genre,
        on=['genre', 'release_week'],
        how='left'
    ).fillna({'movie_count': 0})
    
    prepared_data_for_traces['movie_count'] = prepared_data_for_traces['movie_count'].astype(int)
    prepared_data_for_traces['theta_degrees'] = (prepared_data_for_traces['release_week'] - 0.5) * DEGREES_PER_WEEK
else:
    # Create an empty DataFrame with the correct structure if no focus genres or data
    prepared_data_for_traces = pd.DataFrame(columns=['genre', 'release_week', 'movie_count', 'theta_degrees'])
    # Populate with zeros if MOVIE_GENRES is defined but no data
    if MOVIE_GENRES:
        temp_list = []
        for g in MOVIE_GENRES:
            for w in all_weeks_series:
                temp_list.append({'genre': g, 'release_week': w, 'movie_count': 0, 
                                  'theta_degrees': (w - 0.5) * DEGREES_PER_WEEK})
        prepared_data_for_traces = pd.DataFrame(temp_list)


# --- 1.B: Calculate max R values for dynamic scaling ---
# Max count for each individual genre
max_r_per_genre = {}
if not prepared_data_for_traces.empty:
    max_r_per_genre = prepared_data_for_traces.groupby('genre')['movie_count'].max().to_dict()
    for g in MOVIE_GENRES: # Ensure all genres have an entry, even if 0
        if g not in max_r_per_genre:
            max_r_per_genre[g] = 0

# Max total count for the stacked "Combined" view (sum of counts per week)
max_r_combined = 0
if not prepared_data_for_traces.empty:
    weekly_total_counts = prepared_data_for_traces.groupby('release_week')['movie_count'].sum()
    if not weekly_total_counts.empty:
        max_r_combined = weekly_total_counts.max()

# --- Step 2: Create the Plotly Figure ---
fig = go.Figure()
polar_hole_fraction = 0.05
initial_radial_axis_range = [0, max(10, max_r_combined) * 1.05] # Initial range for combined view

# --- 2.1 Add Seasonal Background Traces ---
for season in SEASONS:
    start_w, end_w = season["start_week"], season["end_week"]
    num_season_weeks = end_w - start_w + 1
    season_theta_center = ((start_w - 1) + num_season_weeks / 2.0) * DEGREES_PER_WEEK
    fig.add_trace(go.Barpolar(
        r=[initial_radial_axis_range[1]], # Use initial max R for season height
        theta=[season_theta_center],
        width=[num_season_weeks * DEGREES_PER_WEEK],
        marker_color=season["color"], marker_line_width=0,
        thetaunit="degrees", name=season["name"], hoverinfo='skip', showlegend=False,
        # Assign to a specific subplot if using subplots, not needed here for background
    ))

# --- 2.2 Add Traces for "Combined Stacked" View AND Individual Genre Views ---
# These traces will be toggled for visibility.
# For the "Combined" view, all these traces will be visible.
# For an individual genre view, only that genre's trace will be visible.

for genre in MOVIE_GENRES:
    genre_data = prepared_data_for_traces[prepared_data_for_traces['genre'] == genre]
    # Ensure genre_data has all weeks, even if count is 0 (already handled by prepared_data_for_traces)
    
    fig.add_trace(go.Barpolar(
        r=genre_data['movie_count'],
        theta=genre_data['theta_degrees'],
        width=[DEGREES_PER_WEEK * 0.9] * len(genre_data),
        thetaunit="degrees",
        name=genre, # This name is crucial for stacking and identification
        marker_color=GENRE_COLOR_MAP.get(genre, '#CCCCCC'),
        customdata=np.stack((genre_data['release_week'], genre_data['movie_count']), axis=-1),
        hovertemplate=( # This hovertemplate will be used for individual view
            f"<b>{genre}</b><br>"
            "Week: %{customdata[0]}<br>"
            "Movies: %{customdata[1]}<extra></extra>"
        ),
        # For the combined (stacked) view, the hover will be on the total bar,
        # so individual segment hovers might be less critical or can be customized later if needed.
        # For now, this hover works for both views.
        visible=True # Initially, all are visible to form the stacked combined plot
    ))

# --- Step 3: Create Dropdown Menu for View Selection ---
dropdown_options = [COMBINED_VIEW_LABEL] + MOVIE_GENRES
buttons = []

num_season_traces = len(SEASONS)
num_genre_traces = len(MOVIE_GENRES) # Each genre has one trace

for i, option_label in enumerate(dropdown_options):
    visibility_mask_for_genres = [False] * num_genre_traces
    new_radial_range = [0, 10] # Default small range
    current_title_suffix = option_label

    if option_label == COMBINED_VIEW_LABEL:
        visibility_mask_for_genres = [True] * num_genre_traces # All genre traces visible for stacking
        new_radial_range = [0, max(10, max_r_combined) * 1.05]
        current_title_suffix = "All Genres (Stacked)"
    else: # Individual genre selected
        genre_index = MOVIE_GENRES.index(option_label)
        visibility_mask_for_genres[genre_index] = True
        genre_max_r = max_r_per_genre.get(option_label, 0)
        new_radial_range = [0, max(10, genre_max_r) * 1.05]

    # Full visibility: seasons + genre_traces
    full_visibility_mask = [True] * num_season_traces + visibility_mask_for_genres
    
    buttons.append(dict(
        label=option_label,
        method="update",
        args=[
            {"visible": full_visibility_mask}, # Update trace visibility
            { # Update layout components
                "title.text": f"Weekly Movie Releases: {current_title_suffix}",
                "polar.radialaxis.range": new_radial_range,
                # Update radial axis for seasonal backgrounds too if they depend on the max range
                # This requires iterating through traces and updating 'r' for seasonal traces
            }
        ],
        # To update season trace heights dynamically, we'd need to specify 'args' for specific traces
        # e.g. args=[{'visible': ..., 'r[0]': [new_r_max_for_season_1], ...}, layout_updates]
        # This becomes complex. A simpler way is to set season 'r' high initially or accept fixed height.
        # For now, season height is fixed based on initial_radial_axis_range.
        # If dynamic season height is critical, it would involve more complex 'args'.
    ))

fig.update_layout(
    barmode='stack', # CRUCIAL for the combined view
    updatemenus=[
        dict(
            buttons=buttons,
            direction="down", pad={"r": 10, "t": 10}, showactive=True,
            x=0.01, xanchor="left", y=1.18, yanchor="top",
            active=0, # "All Genres (Stacked)" is the default
            font=dict(size=11)
        )
    ]
)

# --- Step 4: Style the Plot ---
initial_title = f"Weekly Movie Releases: {COMBINED_VIEW_LABEL}"
tick_weeks = list(range(1, N_WEEKS + 1, 4))
angular_tickvals = [(w - 1) * DEGREES_PER_WEEK for w in tick_weeks]
angular_ticktext = [f"W{w}" for w in tick_weeks]

fig.update_layout(
    title_text=initial_title,
    title_x=0.5, title_font_size=18,
    font_family="Arial, sans-serif", font_size=10,
    polar=dict(
        radialaxis=dict(
            visible=True, showticklabels=True, ticksuffix=' movies',
            range=initial_radial_axis_range, # Set initial range
            gridcolor="#cccccc", gridwidth=0.5, linecolor='black',
            angle=0, tickfont_size=9
        ),
        angularaxis=dict(
            thetaunit="degrees", tickmode='array',
            tickvals=angular_tickvals, ticktext=angular_ticktext,
            direction="clockwise", rotation=90, # W1 at top
            period=360,
            gridcolor="#dddddd", gridwidth=0.5,
            showline=True, linecolor='black',
            tickfont_size=9
        ),
        bgcolor="rgba(255,255,255,0.9)",
        hole=polar_hole_fraction
    ),
    showlegend=True, # Show legend for stacked view to identify colors
    legend=dict(
        traceorder='normal', # Or 'reversed' if you prefer stack order in legend
        font=dict(size=9),
        yanchor="top", y=0.99, # Position legend
        xanchor="left", x=1.01
    ),
    height=750, width=900, # Adjusted width for legend
    paper_bgcolor='white', plot_bgcolor='white',
    margin=dict(t=120, b=50, l=50, r=150) # Increased right margin for legend
)

# General hover label style
fig.update_traces(
    selector=dict(type='barpolar'),
    hoverlabel=dict(
        bgcolor="rgba(255,255,255,0.95)",
        font_size=12, font_family="Arial, sans-serif",
        bordercolor="rgba(100,100,100,0.5)"
    )
)

# --- Step 5: Show the Plot ---
fig.show()

In [45]:
# Save the figure as an HTML file
output_file = "../data/weekly_movie_releases_combined_stacked.html" # Adjust path if needed

In [54]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px

# --- Configuration ---
try:
    from plot_style import MOVIE_GENRES, GENRE_COLOR_MAP
except ImportError:
    print("Warning: plot_style.py not found. Using fallback definitions.")
    MOVIE_GENRES = ['Drama', 'Comedy', 'Documentary', 'Romance', 'Action', 'Crime', 'Thriller', 'Horror', 'Adventure', 'Mystery']
    _COLORS_HEX_FALLBACK = ['#003f5c', '#f9a602', '#7a7a7a', '#ff7b9c', '#ef562f', '#2f4f4f', '#008080', '#8b0000', '#556b2f', '#6a0dad']
    if len(MOVIE_GENRES) > len(_COLORS_HEX_FALLBACK): _COLORS_HEX_FALLBACK = px.colors.qualitative.Plotly[:len(MOVIE_GENRES)]
    GENRE_COLOR_MAP = dict(zip(MOVIE_GENRES, _COLORS_HEX_FALLBACK[:len(MOVIE_GENRES)]))

# --- Constants ---
N_WEEKS = 52
DEGREES_PER_WEEK = 360.0 / N_WEEKS
COMBINED_VIEW_LABEL = "All Genres (Stacked Ring)"
POLAR_HOLE_FRACTION = 0.5 # << Increase this for a larger inner hole (e.g., 0.5 to 0.75)

# --- Season Definitions ---
SEASONS = [
    {"name": "Winter", "start_week": 51, "end_week": N_WEEKS, "color": "rgba(173, 216, 230, 0.1)"}, # Made more transparent
    {"name": "Winter", "start_week": 1, "end_week": 11, "color": "rgba(173, 216, 230, 0.1)"},
    {"name": "Spring", "start_week": 12, "end_week": 24, "color": "rgba(144, 238, 144, 0.1)"},
    {"name": "Summer", "start_week": 25, "end_week": 37, "color": "rgba(255, 255, 153, 0.15)"},
    {"name": "Autumn", "start_week": 38, "end_week": 50, "color": "rgba(255, 192, 122, 0.1)"}
]

# --- Step 1: Load and Prepare Data ---
def create_sample_dataframe():
    num_records = 4000
    all_possible_genres_for_sampling = MOVIE_GENRES + ['Sci-Fi', 'Fantasy', 'Animation', 'Family', 'War', 'Music', 'Biography']
    genres_list = []
    for _ in range(num_records):
        num_genres_for_movie = np.random.choice([1, 2, 3], p=[0.6, 0.3, 0.1])
        selected_genres = np.random.choice(all_possible_genres_for_sampling, num_genres_for_movie, replace=False)
        genres_list.append(", ".join(selected_genres))
    data = {
        'title_id': range(1, num_records + 1), 'genre': genres_list,
        'release_date_full': pd.Series(pd.date_range(start='2022-01-01', end='2023-12-31', freq='D')).sample(num_records, replace=True).values,
    }
    df_temp = pd.DataFrame(data)
    num_na = int(num_records * 0.05)
    na_indices = np.random.choice(df_temp.index, num_na, replace=False)
    df_temp.loc[na_indices, 'release_date_full'] = pd.NaT
    return df_temp

df = create_sample_dataframe()
df['release_date_full'] = pd.to_datetime(df['release_date_full'], errors='coerce')
df.dropna(subset=['release_date_full'], inplace=True)
df['release_week'] = df['release_date_full'].dt.isocalendar().week.astype(int)
df.loc[df['release_week'] > N_WEEKS, 'release_week'] = N_WEEKS

df_exploded = df.assign(genre=df['genre'].astype(str).str.strip().str.split(', ')).explode('genre')
df_exploded['genre'] = df_exploded['genre'].str.strip()
df_focus_genres = df_exploded[df_exploded['genre'].isin(MOVIE_GENRES)].copy()

all_weeks_series = pd.Series(range(1, N_WEEKS + 1), name='release_week')
prepared_data_for_traces = pd.DataFrame()
if not df_focus_genres.empty and MOVIE_GENRES:
    weekly_counts_per_genre = df_focus_genres.groupby(['genre', 'release_week']).size().reset_index(name='movie_count')
    all_genres_weeks_template = pd.MultiIndex.from_product([MOVIE_GENRES, all_weeks_series], names=['genre', 'release_week']).to_frame(index=False)
    prepared_data_for_traces = pd.merge(all_genres_weeks_template, weekly_counts_per_genre, on=['genre', 'release_week'], how='left').fillna({'movie_count': 0})
else: # Create dummy zero data if needed
    temp_list = []
    for g in MOVIE_GENRES:
        for w_val in all_weeks_series:
            temp_list.append({'genre': g, 'release_week': w_val, 'movie_count': 0})
    prepared_data_for_traces = pd.DataFrame(temp_list)

prepared_data_for_traces['movie_count'] = prepared_data_for_traces['movie_count'].astype(int)
prepared_data_for_traces['theta_degrees'] = (prepared_data_for_traces['release_week'] - 0.5) * DEGREES_PER_WEEK

max_r_per_genre = {}
if not prepared_data_for_traces.empty:
    max_r_per_genre = prepared_data_for_traces.groupby('genre')['movie_count'].max().to_dict()
for g in MOVIE_GENRES:
    if g not in max_r_per_genre: max_r_per_genre[g] = 0

max_r_combined = 0
if not prepared_data_for_traces.empty:
    weekly_total_counts = prepared_data_for_traces.groupby('release_week')['movie_count'].sum()
    if not weekly_total_counts.empty: max_r_combined = weekly_total_counts.max()

# --- Step 2: Create the Plotly Figure ---
fig = go.Figure()
# The radialaxis.range will be [0, max_bar_length].
# The '0' of this range corresponds to the edge of the hole.
initial_max_bar_length = max(10, max_r_combined) # Max length of bars extending from hole
initial_radial_axis_range = [0, initial_max_bar_length * 1.05]


# --- 2.1 Add Seasonal Background Traces ---
# For ring plots, seasonal backgrounds can be tricky.
# Option 1: Full radial sectors (as before, but visually different with hole)
# Option 2: Thin rings at specific radii (more complex)
# Let's stick with Option 1, understanding they will also start from the hole's edge.
# Their `r` value will be the max bar length, so they fill the data area.
for season in SEASONS:
    start_w, end_w = season["start_week"], season["end_week"]
    num_season_weeks = end_w - start_w + 1
    season_theta_center = ((start_w - 1) + num_season_weeks / 2.0) * DEGREES_PER_WEEK
    fig.add_trace(go.Barpolar(
        r=[initial_radial_axis_range[1]], # Fill up to the max of the data area from hole
        theta=[season_theta_center],
        width=[num_season_weeks * DEGREES_PER_WEEK],
        marker_color=season["color"], marker_line_width=0,
        thetaunit="degrees", name=season["name"], hoverinfo='skip', showlegend=False
    ))

# --- 2.2 Add Traces for "Combined Stacked" View AND Individual Genre Views ---
for genre in MOVIE_GENRES:
    genre_data = prepared_data_for_traces[prepared_data_for_traces['genre'] == genre]
    fig.add_trace(go.Barpolar(
        r=genre_data['movie_count'], # These are the actual bar lengths
        theta=genre_data['theta_degrees'],
        width=[DEGREES_PER_WEEK * 0.9] * len(genre_data),
        thetaunit="degrees", name=genre,
        marker_color=GENRE_COLOR_MAP.get(genre, '#CCCCCC'),
        customdata=np.stack((genre_data['release_week'], genre_data['movie_count']), axis=-1),
        hovertemplate=(f"<b>{genre}</b><br>Week: %{{customdata[0]}}<br>Movies: %{{customdata[1]}}<extra></extra>"),
        visible=True
    ))

# --- Step 3: Create Dropdown Menu for View Selection ---
dropdown_options = [COMBINED_VIEW_LABEL] + MOVIE_GENRES
buttons = []
num_season_traces = len(SEASONS)
num_genre_traces = len(MOVIE_GENRES)

for i, option_label in enumerate(dropdown_options):
    visibility_mask_for_genres = [False] * num_genre_traces
    new_max_bar_length = 10 # Default small length
    current_title_suffix = option_label

    if option_label == COMBINED_VIEW_LABEL:
        visibility_mask_for_genres = [True] * num_genre_traces
        new_max_bar_length = max(10, max_r_combined)
        current_title_suffix = "All Genres (Stacked Ring)"
    else:
        genre_index = MOVIE_GENRES.index(option_label)
        visibility_mask_for_genres[genre_index] = True
        genre_max_r = max_r_per_genre.get(option_label, 0)
        new_max_bar_length = max(10, genre_max_r)

    new_radial_range_for_bars = [0, new_max_bar_length * 1.05]
    full_visibility_mask = [True] * num_season_traces + visibility_mask_for_genres
    
    # Dynamic update for seasonal trace 'r' values to match current data scale
    seasonal_r_updates = {}
    for s_idx in range(num_season_traces):
        seasonal_r_updates[f'r[{s_idx}]'] = [[new_radial_range_for_bars[1]]] # Note: r needs to be a list

    buttons.append(dict(
        label=option_label,
        method="update",
        args=[
            {
                "visible": full_visibility_mask,
                **seasonal_r_updates # Add the 'r' updates for seasonal traces
            },
            {
                "title.text": f"Weekly Movie Releases: {current_title_suffix}",
                "polar.radialaxis.range": new_radial_range_for_bars,
            }
        ]
    ))

fig.update_layout(
    barmode='stack',
    updatemenus=[
        dict(
            buttons=buttons, direction="down", pad={"r": 10, "t": 10}, showactive=True,
            x=0.01, xanchor="left", y=1.18, yanchor="top", active=0, font=dict(size=11)
        )
    ]
)

# --- Step 4: Style the Plot ---
initial_title = f"Weekly Movie Releases: {COMBINED_VIEW_LABEL}"
tick_weeks = list(range(1, N_WEEKS + 1, 4))
angular_tickvals = [(w - 1) * DEGREES_PER_WEEK for w in tick_weeks]
angular_ticktext = [f"W{w}" for w in tick_weeks]

fig.update_layout(
    title_text=initial_title, title_x=0.5, title_font_size=18,
    font_family="Arial, sans-serif", font_size=10,
    polar=dict(
        hole=POLAR_HOLE_FRACTION, # <<< SET THE HOLE SIZE HERE
        radialaxis=dict(
            visible=True, showticklabels=True, ticksuffix=' movies',
            range=initial_radial_axis_range, # This range is for the bar lengths from the hole's edge
            gridcolor="#cccccc", gridwidth=0.5, linecolor='black',
            angle=0, tickfont_size=9,
            # tickmode='linear', # Or auto, might give better ticks for this style
            # nticks=5 # Suggest number of ticks
        ),
        angularaxis=dict(
            thetaunit="degrees", tickmode='array',
            tickvals=angular_tickvals, ticktext=angular_ticktext,
            direction="clockwise", rotation=90, period=360,
            gridcolor="#dddddd", gridwidth=0.5, showline=True, linecolor='black',
            tickfont_size=9
        ),
        bgcolor="rgba(255,255,255,0.0)" # Make polar area transparent to see paper_bgcolor
    ),
    showlegend=True,
    legend=dict(traceorder='normal', font=dict(size=9), yanchor="top", y=0.99, xanchor="left", x=1.01),
    height=750, width=900,
    paper_bgcolor='white', # Main background
    # plot_bgcolor='rgba(0,0,0,0)', # Ensure plot area itself is transparent if polar.bgcolor is also transparent
    margin=dict(t=120, b=50, l=50, r=150)
)

fig.update_traces(selector=dict(type='barpolar'), hoverlabel=dict(
    bgcolor="rgba(255,255,255,0.95)", font_size=12,
    font_family="Arial, sans-serif", bordercolor="rgba(100,100,100,0.5)"
))

# --- Step 5: Show the Plot ---
fig.show()

In [52]:
# Save the figure as an HTML file
output_file = "../figures/weekly_movie_releases_combined_stacked.html" # Adjust path if needed
fig.write_html(output_file, include_plotlyjs='cdn')
print(f"Figure saved to {output_file}")

Figure saved to ../figures/weekly_movie_releases_combined_stacked.html
